<a href="https://colab.research.google.com/github/jodejetalo99/Introduccion-al-Aprendizaje-Profundo/blob/main/IPA_T1_E3_JJTL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Universidad Nacional Autónoma de México
# Instituto de Investigaciones en Matemáticas Aplicadas y en Sistemas
# Introducción al Aprendizaje Profundo
# José de Jesús Tapia López
# Tarea 1: Preceptrón y Redes Densas
# 19 de Marzo del 2021

## Ejercicio 3

3. Entrena una red completamente conectada para aproximar la compuerta XOR.

In [1]:
import random
# redes neuronales
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as T
# algebra lineal
import numpy as np

In [2]:
def set_seed(seed=171299):
    """Initializes pseudo-random number generators."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    
# reproducibilidad
set_seed()

La siguiente función es para la salida de la XOR. Como podremos observar más adelante la compuerta XOR, al final se le aplica una función sigmoide, con el objetivo de separar las etiquetas de 0 y 1. Si al aplicar por último la función sigmoide, el valor es mayor a 0.5, la etiqueta es 1, y si es menor o igual a 0.5, la etiqueta es cero.(Se está viendo como probabilidades)

In [3]:
def step(z):
    """Computes step function."""
    return 1.0 if z > 0.5 else 0.0

In [4]:
# vectorizamos la función para que
# aplique a un arreglo de entradas
step_vec = np.vectorize(step)

In [5]:
class XOR(nn.Module):
    
    def __init__(self):   
        """
        Inicialización de las capas del modelo
        """
        super(XOR, self).__init__()
        self.fc1 = nn.Linear(2,2) 
        self.fc2 = nn.Linear(2,1)
     
    def forward(self,x):
        """
        Definición de la estructura del modelo 
        (se aplica como función de activación una sigmoide)
        Parámetros: 
        x (tensor): entrada del modelo
        Devuelve: 
        y_pred: predicción
        """
        x_in = torch.sigmoid(self.fc1(x)) 
        y_pred = torch.sigmoid(self.fc2(x_in)) 
        return y_pred
    
xor = XOR()

# Datos de entrenamiento
x = torch.tensor([[0.,0.],[0.,1.],[1.,0.],[1.,1.]])
y = torch.tensor([[0.],[1.],[1.],[0.]])

# Usamos el error cuadrático medio como criterio
criterio = nn.MSELoss()                            
# optimizador Adam (método de optimización estocástica) modifica la tasa de aprendizaje
optimizador = optim.Adam(xor.parameters(), lr=0.1)  

epocas = 1000

for e in range(epocas):
    # Establece los gradientes de todos los tensores optimizados a cero.
    optimizador.zero_grad()
    # salida de la XOR    
    salida = xor(x)

    # forward, backward y optimización

    # función de perdida basada en el criterio
    perdida = criterio(salida, y)
    if e % 100 == 0:
      print("Época:{0}, Pérdida: {1}".format(e,perdida))
    # Los gradientes son "almacenados" por los propios tensores (
    # una vez que llama backward() en la pérdida 
    perdida.backward()
    # hace que el optimizador itere sobre todos los parámetros (tensores) 
    # que se supone debe actualizar
    optimizador.step() 

print("\nCriterio: MSE,  Optimizador: Adam", "lr = 0.1")
prueba = xor(x)
print('\nx_1 \tx_2 \ty\ty_hat')
print('-----------------------------')
for i in range(x.shape[0]):
    x1, x2 = x[i]
    y_hat = step_vec(prueba.detach())
    print(f'{x1}\t{x2}\t{y[i][0]}\t{y_hat[i][0]}')

Época:0, Pérdida: 0.2615337371826172
Época:100, Pérdida: 0.005222637206315994
Época:200, Pérdida: 0.0011664106277748942
Época:300, Pérdida: 0.0006026178598403931
Época:400, Pérdida: 0.00037941010668873787
Época:500, Pérdida: 0.0002648420922923833
Época:600, Pérdida: 0.0001970729645108804
Época:700, Pérdida: 0.00015317954239435494
Época:800, Pérdida: 0.00012288903235457838
Época:900, Pérdida: 0.00010098466009367257

Criterio: MSE,  Optimizador: Adam lr = 0.1

x_1 	x_2 	y	y_hat
-----------------------------
0.0	0.0	0.0	0.0
0.0	1.0	1.0	1.0
1.0	0.0	1.0	1.0
1.0	1.0	0.0	0.0


Podemos observar que esta red aproxima bien la compuerta XOR prácticamente a partir de la época 100 con una pérdida que ya empieza a acercarse mucho a 0. Sin embargo, se intentó usar como optimizador el SGD pero requería de muchas más épocas (más de 10000) para aproximar bien a dicha compuerta (la pérdida disminuía más lentamente):

![](https://raw.githubusercontent.com/jodejetalo99/Introduccion-al-Aprendizaje-Profundo/main/Figuras/T1/IPA_T1_E3_1.PNG)


Más aún, con una tasa de aprendizaje (lr) de 0.01, tanto en el optimizador Adam como en el SGD se tardaban más en disminuir la pérdida y la red no aproximaba bien la compuerta XOR:

![](https://raw.githubusercontent.com/jodejetalo99/Introduccion-al-Aprendizaje-Profundo/main/Figuras/T1/IPA_T1_E3_4.PNG)
![](https://raw.githubusercontent.com/jodejetalo99/Introduccion-al-Aprendizaje-Profundo/main/Figuras/T1/IPA_T1_E3_3.PNG)

Aún así, en las imágenes anteriores observamos que el optimizador ADAM es más eficiente, pues la pérdida la disminuye más rápido que con SGD.


Creo que esto se debe a que las tasas de aprendizaje más pequeñas requieren más épocas de entrenamiento (requiere más tiempo para entrenar) debido a los cambios más pequeños realizados en los pesos en cada actualización, mientras que las tasas de aprendizaje más grandes dan como resultado cambios rápidos y requieren menos épocas de entrenamiento. Sin embargo, las tasas de aprendizaje más altas a menudo dan como resultado un conjunto final de pesos subóptimo. Por ello, creo que en principio un lr de 0.1 fue un número adecuado.